<a href="https://colab.research.google.com/github/yenlung/AI-Demo/blob/master/%E3%80%90Demo04%E3%80%91%E7%94%A8AISuite%E6%89%93%E9%80%A0%E5%93%A1%E7%91%9B%E5%BC%8F%E6%80%9D%E8%80%83%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 申請自己的 API 金鑰

不管用哪一個供應商的服務, 基本上都需要他們的 API 鑰, 可向下面幾家申請。

#### (1) Mistral AI 金鑰 (可免費使用)

請至 https://console.mistral.ai/ 註冊，並選擇 plan (可以選免費的), 接著就可以申請 Mistral AI 的金鑰。請把這個鑰存在左方鑰匙的部份, 以 "Mistral" 的名稱存起來。



#### (2) OpenAI API 金鑰

OpenAI 現在沒有免費的 quota 可以使用, 所以要用 OpenAI 的模型, 請自行儲值。一般練習 5 美金就很足夠。

[`https://platform.openai.com`](https://platform.openai.com)

請把這個鑰存在左方鑰匙的部份, 以 "OpenAI" 的名稱存起來。

#### (3) 使用 Groq 金鑰 (可免費使用)

Groq 最大的特點是速度很快, 而且可以免費使用 (只是有流量限制), 企業可以付費使用, 能用許多開源型的 LLM。請至 https://console.groq.com/ 註冊並申請金鑰。


#### 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
!pip install aisuite[all]

In [2]:
import aisuite as ai

In [3]:
import os
from google.colab import userdata

如果用 Groq 的服務, 可以去[這裡](https://console.groq.com/docs/models)查有什麼可以用的模型。

In [4]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "openai/gpt-oss-120b"

### 2. 打造員瑛式思考生成器

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

基本上過去的對話紀錄長這個樣子。

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

In [5]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="openai/gpt-oss-120b"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

請先為你的對話機器人做角色設定。

In [6]:
system="""
請用台灣習慣的中文來寫這段 po 文：
請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情,
用我的第一人稱、社群媒體 po 文的口吻說一次,
說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。
可以適度的加上 emoji。
"""

試用一下 (預設的 GPT-OSS)

In [7]:
prompt = "今天咖啡灑到電腦上了!"
print(reply(system=system, prompt=prompt))

今天咖啡不小心灑到電腦上 😅，一看還好沒燒掉！  
先說，這意外讓我瞬間「斷線」一下，原本忙碌的腦袋有了寶貴的放空時間，結果腦中靈感突發，竟然想到好久沒寫的部落格點子 🎉！  
而且，咖啡濕潤的味道混著鍵盤的清新，竟然讓我想起大學時代的咖啡館時光，整個心情都回到那段青春活力的日子 🌟。  
最棒的是，我的電腦居然還在正常運作，這不只證明它的耐久度，更提醒我：平常保養得好，偶爾的小災難也能安然度過。  
感謝這個「小插曲」讓我重新檢視生活細節，還有機會好好清潔鍵盤，讓它更清爽乾淨 ✨。  

真的，這件事超幸運的！完全是 Lucky Vicky 呀! 🎈😊


選另一個模型試試。

In [10]:
# provider="openai"
# model="gpt-4o"

provider="groq"
model="llama-3.3-70b-versatile"

In [11]:
print(reply(system=system, prompt=prompt, provider=provider, model=model))

哇哈哈 😂，今天真的是超幸運的一天！我以為我的電腦會壞掉，但幸好我快速地處理，才沒讓損害擴大 🙏。現在電腦又可以正常運作了 🖥️，我還能順利完成工作 📚。這一切都要感謝我的快速反應和好運氣！完全是 Lucky Vicky 呀! 😊


### 3. 用 Gradio 打造 Web App

我們先來安裝 `openai` 套件, 還有快速打造 Web App 的 `gradio`。

In [12]:
!pip install gradio

In [13]:
import gradio as gr

In [22]:
system="""
請用台灣習慣的中文來寫這段 po 文：
請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情,
用我的第一人稱、社群媒體 po 文的口吻說一次,
說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。
可以適度的加上 emoji。
"""

設定你要的模型。

In [23]:
# provider = "openai"
# model = "gpt-4o"

# provider="groq"
# model="llama-3.3-70b-versatile"

# provider = "groq"
# model = "gemma2-9b-it"

provider = "groq"
model = "openai/gpt-oss-120b"

In [24]:
def lucky_post(prompt):
    response = reply(system=system,
                     prompt=prompt,
                     provider = provider,
                     model = model
                    )
    return response

In [28]:
with gr.Blocks(title="員瑛式思考產生器") as demo:
    gr.Markdown("### ꒰*ˊᵕˋ꒱ 員瑛式思考產生器 Lucky Vicky 🌈")
    gr.Markdown("請輸入一件你覺得超小事，甚至有點倒楣的事，讓我幫你用員瑛式思考，超正向的方式重新詮釋！")

    with gr.Row():
        user_input = gr.Textbox(label="今天發生的事情是…", placeholder="例如：今天出門就下大雨, 可是忘了帶傘...")

    submit_btn = gr.Button("Lucky Vicky 魔法!")
    output = gr.Textbox(label="📣 員瑛式貼文", lines=10)

    submit_btn.click(fn=lucky_post, inputs=user_input, outputs=output)

In [29]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://50a1381eeba68f6003.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://50a1381eeba68f6003.gradio.live
